# Hands-on session with pke - part 2

This notebook provides a series of examples on how to parameterize the keyphrase extraction models in `pke`.
More specifically, we will see how to customize the identification of keyphrase candidates and how to use different models implemented in `pke`.

In a second time, we will conduct a series of experiments to compare several models on Inspec, a commonly-used benchmark dataset for keyphrase extraction that contains bibliographic records (i.e. title/abstract from scientific papers).

As a reminder, `pke` provides a standardized API for extracting keyphrases from a document by typing the following 5 lines:

```python
import pke

extractor = pke.unsupervised.TfIdf()        # initialize a keyphrase extraction model, here TFxIDF
extractor.load_document(input='text')       # load the content of the document (file or str)
extractor.candidate_selection()             # identify keyphrase candidates
extractor.candidate_weighting()             # weight keyphrase candidates
keyphrases = extractor.get_n_best(n=5)      # select the 5-best candidates as keyphrases
```

## Preamble on keyphrase extraction datasets using 🤗 datasets

For simplicity and ease of use, we rely on the `datasets` module from 🤗 huggingface to load and access sample documents.



In [11]:
from datasets import load_dataset

# load the inspec dataset
dataset = load_dataset('boudinfl/inspec', "all")

# let's have a look at one sample document from the validation split
sample = dataset["validation"][233]

print("id: {}".format(sample["id"]))
print("title: {}...".format(sample["title"][:50]))
print("abstract: {}...".format(sample["abstract"][:50]))
print("controlled keyphrases: {}; ...".format("; ".join(sample["contr"][:3])))
print("uncontrolled keyphrases: {}; ...".format("; ".join(sample["uncontr"][:3])))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset inspec downloaded and prepared to /Users/boudin-f/.cache/huggingface/datasets/boudinfl___inspec/all/1.0.1/f333b3e8c7190f09ecbc2eee2706f13dd7370a0f3d72bb15ceb6e34ee90a6aa7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

id: 1895
title: An algorithm combining neural networks with fundam...
abstract: An algorithm combining neural networks with the fu...
controlled keyphrases: chromium alloys; iron alloys; neural nets; ...
uncontrolled keyphrases: algorithm; neural networks; fundamental parameters; ...


## Model parameterization - candidate selection

Candidate selection is a crucial stage in keyphrase extraction as it determines the size of the search space (i.e. number of candidates to rank/weight) and the upper bound performance (i.e. maximum recall).
Here, we will see how to configure the candidate selection method in `pke` to achieve the best compromise between search space and maximum performance.

In order to compare candidate selection methods, we compute the maximum recall score against the gold standard (human-assigned) keyphrases as

$$max\_recall = \frac{| candidates \cap references|}{|references|}$$

In [39]:
import pke

# initialize a simple model that ranks candidates using their position
extractor = pke.unsupervised.FirstPhrases()

# the text to process is the title concatenated to the abstract
text = sample["title"] + ". " + sample["abstract"]

# the references in stemmed form to compute the maximum recall
references = sample["uncontr_stems"]

# load the document using the initialized model
extractor.load_document(input=text, language='en')

### Setting up a linguistic-based selection method

In [45]:
grammar = r"""
                NP:
                    {<ADJ>*<NOUN|PROPN>+}
            """

extractor.grammar_selection(grammar=grammar)

# let's see how many candidates are identified
print("{} keyphrase candidates were identified".format(len(extractor.candidates)))

# print out a sample
candidates = [*extractor.candidates]
print("- Subsample of candidates:", ' ; '.join(candidates[:5]))

# compute the maximum recall
max_recall = len(set(references) & set(candidates)) / len(set(references))
print("- Maximum recall: {:.3f}".format(max_recall))

# identify missed reference keyphrases
missed = set(references) - set(candidates)
print("- Missed reference keyphrases: {}".format(missed))

33 keyphrase candidates were identified
- Subsample of candidates: algorithm ; neural network ; fundament paramet ; fundament paramet equat ; nnfp
- Maximum recall: 0.765
- Missed reference keyphrases: {'theoret correct model', 'lachance-trail model', 'x-ray fluoresc analysi', 'nonlinear matrix effect'}


<span style="background:lightpink">
    Exercice: try modifying/adding PoS patterns of the grammar to increase the maximum recall.
</span>

### Setting up a n-gram-based selection method

In [46]:
# first we need to empty the candidates
extractor.candidates.clear()

# here we use a simple n-gram selection for candidates
extractor.ngram_selection(n=3)

# let's see how many candidates are identified
print("{} keyphrase candidates were identified".format(len(extractor.candidates)))

# print out a sample
candidates = [*extractor.candidates]
print("- Subsample of candidates:", ' ; '.join(candidates[:5]))

# compute the maximum recall
max_recall = len(set(references) & set(candidates)) / len(set(references))
print("- Maximum recall: {:.3f}".format(max_recall))

# identify missed reference keyphrases
missed = set(references) - set(candidates)
print("- Missed reference keyphrases: {}".format(missed))

441 keyphrase candidates were identified
- Subsample of candidates: an ; an algorithm ; an algorithm combin ; algorithm ; algorithm combin
- Maximum recall: 0.824
- Missed reference keyphrases: {'lachance-trail model', 'x-ray fluoresc analysi', 'nonlinear matrix effect'}


<span style="background:lightpink">
    Exercice: try removing unrelevant candidates to reduce the search space.
</span>

## Model parameterization - candidate weighting/ranking

The keyphrase extraction model that we use in `pke` define how candidates are weighted. For example, in TopicRank, candidates are weighted using a graph-based ranking model whereas in Yake, candidates are weighted using a combination of statistical features (e.g. position, frequency). Here, we will see how to use different models implemented in `pke`. For comparison purposes, we will use a unified candidate selection method (as presented above). Models are evaluated against the gold standard (human-assigned) keyphrases by computing the precision, recall and f-measure at the top-N extracted keyphases as:

$$ precision@N = \frac{| top-N candidates \cap references|}{|top-N candidates|} $$

$$ recall@N = \frac{| top-N candidates \cap references|}{|references|} $$

$$ f-measure@N = 2 \times \frac{precision@N \cdot recall@N }{precision@N + recall@N} $$

In [52]:
# the text to process is the title concatenated to the abstract
text = sample["title"] + ". " + sample["abstract"]

# the unified grammar for candidate selection
grammar="NP: {<ADJ>*<NOUN|PROPN>+}"

### Baseline model: TopicRank

In [59]:
extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=text, language='en')
extractor.grammar_selection(grammar=grammar)
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=5, stemming=True)

top5 = [candidate for candidate, weight in keyphrases]
print("top-5 keyphrases:", '; '.join(top5))

# evaluate the Precision / Recall / F-measure of the model
P = len(set(top5) & set(references)) / len(top5)
R = len(set(top5) & set(references)) / len(references)
F = 2 * (P*R) / (P+R)
print("P@5: {:.3f} R@5: {:.3f} F@5: {:.3f}".format(P, R, F))

top-5 keyphrases: fundament paramet; nnfp; classic theoret correct model; algorithm; non - linear matrix effect
P@5: 0.400 R@5: 0.118 F@5: 0.182


### Good performance graph-based ranking model: MultipartiteRank

In [61]:
extractor = pke.unsupervised.MultipartiteRank()
extractor.load_document(input=text, language='en')
extractor.grammar_selection(grammar=grammar)
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=5, stemming=True)

top5 = [candidate for candidate, weight in keyphrases]
print("top-5 keyphrases:", '; '.join(top5))

# evaluate the Precision / Recall / F-measure of the model
P = len(set(top5) & set(references)) / len(top5)
R = len(set(top5) & set(references)) / len(references)
F = 2 * (P*R) / (P+R)
print("P@5: {:.3f} R@5: {:.3f} F@5: {:.3f}".format(P, R, F))

top-5 keyphrases: fundament paramet; algorithm; neural network; nnfp; classic theoret correct model
P@5: 0.600 R@5: 0.176 F@5: 0.273


### Supervised model: Kea

In [60]:
extractor = pke.supervised.Kea()
extractor.load_document(input=text, language='en')
extractor.grammar_selection(grammar=grammar)
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=5, stemming=True)

top5 = [candidate for candidate, weight in keyphrases]
print("top-5 keyphrases:", '; '.join(top5))

# evaluate the Precision / Recall / F-measure of the model
P = len(set(top5) & set(references)) / len(top5)
R = len(set(top5) & set(references)) / len(references)
F = 2 * (P*R) / (P+R)
print("P@5: {:.3f} R@5: {:.3f} F@5: {:.3f}".format(P, R, F))

top-5 keyphrases: nnfp algorithm; neural network; fundament paramet; fundament paramet equat; non - linear matrix effect
P@5: 0.800 R@5: 0.235 F@5: 0.364


/Users/boudin-f/Documents/GitHub/hands-on-with-pke/venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.20.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


<span style="background:lightpink">
    Exercice: try using another model.
</span>

## Benchmarking models on the inspec dataset